<a href="https://colab.research.google.com/github/TanmayKumar-EngStud/Question_Answering_Model/blob/main/New_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting the Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import re , requests
#First: We are just scrape the data from the website
from bs4 import BeautifulSoup

In [10]:
BaseSite ='https://en.wikipedia.org/wiki/Outline_of_health'
data = requests.get(BaseSite)

LinksList= re.findall('(?<=<a href=")/wiki/[A-Za-z ]+(?=")',data.text)

unique = set(LinksList)
LinksList = list(unique)

## **This function is for Refining the overall Text**




In [11]:
def refine(ListText):
    inside = False
    newText = ''
    for text in ListText:
        for word in text:
            if word == '<':
                inside =True
            elif word == '>':
                inside = False
                continue
            if not inside:
                newText = newText + word
                #This must eleminate all the attributes
    newText1 = ''
    for letter in newText:
        if letter == '[' or letter == '(':
            inside = True
        elif letter == ']' or letter == ')':
            inside = False
            continue
        if not inside:
            newText1 = newText1 + letter

    return newText1


In [12]:
def clean(text):
    temp = re.findall("(?<=<p>).*(?=</)",str(text))
    return temp

## Making the model

In [13]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
    def on_epoch_end(self,model):
        loss = model.get_latest_training_loss()

        if self.epoch == 0:
            print(f"Loss after epoch {self.epoch}: {loss}")
        elif self.epoch%100 == 0:
            print(f"Loss after epoch {self.epoch}: {loss - self.loss_previous_step}")
        self.epoch+=1
        self.loss_previous_step = loss

#### **Function for making the list of sentences**

In [14]:
def makeSentence(Paragraph):
    Sentences = []
    for sentence in Paragraph.split('. '):
        Sentences.append(sentence)
    return Sentences

### ***For the generation of the first Model***

In [15]:
#This is the one time code 
def buildFirst_Model(text):
    w2v = Word2Vec(size = 100,
                sg = 1,
                min_count = 1,  
                window = 5,
                workers = 3)
    #initializing the word2vec for the first time



    #Now building up the vocabulary
    sentences = makeSentence(text)
    w2v.build_vocab(sentences)

    w2v.train(sentences,
                total_examples=w2v.corpus_count,
                epochs =1001,
                report_delay=1,
                compute_loss=True,
                callbacks=[callback()])
    w2v.save(f'/content/drive/MyDrive/NLP Work flow/Trained_Models/w2vModel_Version{1}.h5')

### **When the first model is generated then we have to load it and simplimg retrain it.**
<hr>

**Key Features:** <br>
 

*   This will modify the alrady generated Word2Vector model.
*   This are versatile method and will be really easy for retraining and refining the model in a dynamic way.
*   Every modification of the model saves the new version of that model.
*   Every iteration brings the latest model for retraining.




In [16]:
import os
def findLatestModel(path):
    ListofFiles = os.listdir(path)
    if ListofFiles == None:
        return None
    return ListofFiles[-1]

In [17]:
def Modify_Model(text):
    mainPath = '/content/drive/MyDrive/NLP Work flow/Trained_Models/'
    fileName = findLatestModel(mainPath)
    Lw2v = Word2Vec.load(mainPath+fileName)
    num = int(re.findall('(?<=Model_Version)[0-9]+(?=.h5)', fileName)[0])
    sentence = makeSentence(text)
    print(f'Doing Model traing; Iteration: {num +1}')
    Lw2v.build_vocab(sentence, update = True)
    token_count = sum([len(se) for se in sentence])

    print(f'\n\nTHIS IS TOKEN COUNT!! : {token_count}')
    Lw2v.train(sentence,
                epochs =1001,
                total_words = token_count,
                report_delay=1,
                compute_loss=True,
                callbacks=[callback()])
    print(f'Done Model training; Iteration:{num+1}')
    Lw2v.train(f'{mainPath}w2vModel_Version{num+1}.h5')

In [18]:
initial = True
for RawLink in LinksList:
    
    link= "https://en.wikipedia.org/"+RawLink
    temp = requests.get(link)
    soup = BeautifulSoup(temp.content, "html.parser")
    text= clean(soup)
    text= refine(text)
    if initial :
        buildFirst_Model(text)
        initial = False
    else :
        Modify_Model(text)
   
#Achieved all the rough data

Loss after epoch 100: 3780.65625
Loss after epoch 200: 3946.125
Loss after epoch 300: 3806.375
Loss after epoch 400: 3570.75
Loss after epoch 500: 3656.625
Loss after epoch 600: 3889.25
Loss after epoch 700: 3930.5
Loss after epoch 800: 4224.0
Loss after epoch 900: 3537.0
Loss after epoch 1000: 4386.75
Doing Model traing; Iteration: 2


THIS IS TOKEN COUNT!! : 22276
Loss after epoch 100: 28881.75
Loss after epoch 200: 23429.5
Loss after epoch 300: 22598.0
Loss after epoch 400: 18917.0
Loss after epoch 500: 19468.0
Loss after epoch 600: 19062.0
Loss after epoch 700: 19172.0
Loss after epoch 800: 20460.0
Loss after epoch 900: 21164.0
Loss after epoch 1000: 20574.0
Done Model training; Iteration:2


ValueError: ignored